# Assignment: Linear Models
## Foundations of Machine Learning
## Do Q1 and one other question

**Q1.** Load `./data/Q1_clean.csv`. The data include

- `Price` per night
- `Review Scores Rating`: The average rating for the property
- `Neighbourhood `: The bourough of NYC. Note the space, or rename the variable.
- `Property Type`: The kind of dwelling
- `Room Type`: The kind of space being rented

1. Compute the average prices and scores by `Neighbourhood `; which bourough is the most expensive on average? Create a kernel density plot of price and log price, grouping by `Neighbourhood `.


```
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

df = pd.read_csv('/Users/tamerafang/PycharmProjects/DS3002_LinearRegression/Q1_clean.csv')
df.head()
df.loc[:,['Price','Neighbourhood '] ].groupby('Neighbourhood ').describe()
sns.kdeplot(x=df['Price'], hue=df['Neighbourhood '])
plt.show()
sns.kdeplot(x=np.log(df['Price']), hue=df['Neighbourhood '])
plt.show()
```
* Manhattan is the most expensive.


2. Regress price on `Neighbourhood ` by creating the appropriate dummy/one-hot-encoded variables, without an intercept in the linear model and using all the data. Compare the coefficients in the regression to the table from part 1. What pattern do you see? What are the coefficients in a regression of a continuous variable on one categorical variable?


```
from sklearn import linear_model
y = df['Price']
x = pd.get_dummies(df['Neighbourhood '], dtype='int')
reg = linear_model.LinearRegression(fit_intercept=False).fit(x, y)
results = pd.DataFrame({'variable':reg.feature_names_in_, 'coefficient': reg.coef_})
print(results)
```
* The coefficients from part 1 are the same now, showing a pattern that the regression coefficients effectively represent the average prices for each neighbourhood. For a continuous variable on one categorical variable, the coefficients for each category of the categorical variable directly reflect the mean value of the continuous variable for that category.



3. Repeat part 2, but leave an intercept in the linear model. How do you have to handle the creation of the dummies differently? What is the intercept? Interpret the coefficients. How can I get the coefficients in part 2 from these new coefficients?
```
from sklearn import linear_model
y = df['Price']
x = pd.get_dummies(df['Neighbourhood '], dtype='int', drop_first = True)
reg = linear_model.LinearRegression().fit(x, y)
results = pd.DataFrame({'variable':reg.feature_names_in_, 'coefficient': reg.coef_})
print(results)
print(reg.intercept_)
results = pd.DataFrame({'variable':reg.feature_names_in_, 'coefficient': reg.coef_+reg.intercept_})
print(results)
```
* When including an intercept in the linear model, you must drop one of the dummy variables created from the 'Neighbourhood' category to avoid multicollinearity and establish a baseline comparison. In this case, the Bronx was dropped. The Brox becomes the reference category and its coefficient from the previous regression becomes the intercept. This makes all the coefficients for this regression are now relative to the Bronx. To get the coefficients from part 2 from these new coefficients, we can add the regression coefficient values to the intercept.


4. Split the sample 80/20 into a training and a test set. Run a regression of `Price` on `Review Scores Rating` and `Neighbourhood `. What is the $R^2$ and RMSE on the test set? What is the coefficient on `Review Scores Rating`? What is the most expensive kind of property you can rent?
```
from sklearn import linear_model
from sklearn.model_selection import train_test_split
y = df['Price']
x = df.loc[:, ['Review Scores Rating', 'Neighbourhood ']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=100)
z_train = pd.concat([x_train['Review Scores Rating'], pd.get_dummies(x_train['Neighbourhood '], dtype='int')], axis = 1)
z_test = pd.concat([x_test['Review Scores Rating'], pd.get_dummies(x_test['Neighbourhood '], dtype='int')], axis = 1)
reg = linear_model.LinearRegression(fit_intercept=False).fit(z_train, y_train)
y_hat = reg.predict(z_test)
print('Rsq: ', reg.score(z_test, y_test))
rmse = np.sqrt( np.mean( (y_test - y_hat)**2 ))
print('RMSE: ', rmse)
results = pd.DataFrame({'variable':reg.feature_names_in_, 'coefficient': reg.coef_})
print(results)
answer = 100*1.032257 + 89.4
print(answer)
```
* The Rsq is 0.06701086106947296 and the RMSE is 125.01092061382933. The coefficient on Review Scores Rating is 1.032257. To find the most expensive kind of property you can rent, you would take the calculation of 100*1.032257 + 89.4 to get the answer 192.6257.

5. Split the sample 80/20 into a training and a test set. Run a regression of `Price` on `Review Scores Rating` and `Neighbourhood ` and `Property Type`. What is the $R^2$ and RMSE on the test set? What is the coefficient on `Review Scores Rating`? What is the most expensive kind of property you can rent?
```
from sklearn import linear_model
from sklearn.model_selection import train_test_split
y = df['Price']
x = df.loc[:, ['Review Scores Rating', 'Neighbourhood ', 'Room Type']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=100)
z_train = pd.concat([x_train['Review Scores Rating'], pd.get_dummies(x_train['Neighbourhood '], dtype='int'), pd.get_dummies(x_train['Room Type'], dtype='int')], axis = 1)
z_test = pd.concat([x_test['Review Scores Rating'], pd.get_dummies(x_test['Neighbourhood '], dtype='int'), pd.get_dummies(x_test['Room Type'], dtype='int')], axis = 1)
reg = linear_model.LinearRegression(fit_intercept=False).fit(z_train, y_train)
y_hat = reg.predict(z_test)
print('Rsq: ', reg.score(z_test, y_test))
rmse = np.sqrt( np.mean( (y_test - y_hat)**2 ))
print('RMSE: ', rmse)
results = pd.DataFrame({'variable':reg.feature_names_in_, 'coefficient': reg.coef_})
print(results)
answer = 110.617+53.69+100*.0626
print(answer)
```
* The Rsq is 0.2203534812928234 and the RMSE is 114.27692123130632. The coefficient on Review Scores Rating is 0.626912. To find the most expensive kind of property you can rent, you would take the calculation of 110.617+53.69+100*.0626 to get the answer 170.567.


6. What does the coefficient on `Review Scores Rating` mean if it changes from part 4 to 5? Hint: Think about how multilple linear regression works.
* The coefficient of Review Scores Rating changes in part 4 and 5, as it goes from 1.032257 in part 4 and then to 0.626912 in part 5. This change means there's been an inclusion of additional variables. We added Room Type which can contribute to the variability in room prices. Once we are able to control for room type, the other variables become less powerful predictors so the coefficient decreases.


7. (Optional) We've included `Neighborhood ` and `Property Type` separately in the model. How do you interact them, so you can have "A bedroom in Queens" or "A townhouse in Manhattan". Split the sample 80/20 into a training and a test set and run a regression including that kind of "property type X neighborhood" dummy, plus `Review Scores Rating`. How does the slope coefficient for `Review Scores Rating`, the $R^2$, and the RMSE change? Do they increase significantly compares to part 5? Are the coefficients in this regression just the sum of the coefficients for `Neighbourhood ` and `Property Type` from 5? What is the most expensive kind of property you can rent?


**Q2.** This question is a case study for linear models. The data are about car prices. In particular, they include:

  - `Price`, `Color`, `Seating_Capacity`
  - `Body_Type`: crossover, hatchback, muv, sedan, suv
  - `Make`, `Make_Year`: The brand of car and year produced
  - `Mileage_Run`: The number of miles on the odometer
  - `Fuel_Type`: Diesel or gasoline/petrol
  - `Transmission`, `Transmission_Type`:  speeds and automatic/manual

  1. Load `cars_hw.csv`. These data were really dirty, and I've already cleaned them a significant amount in terms of missing values and other issues, but some issues remain (e.g. outliers, badly scaled variables that require a log or arcsinh transformation). Clean the data however you think is most appropriate.


```
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

df = pd.read_csv('/Users/tamerafang/PycharmProjects/DS3002_LinearRegression/cars_hw.csv')
df0 = df
sns.boxplot(data=df)
plt.show()
print(df.columns)
df['price_ihs'] = np.arcsinh(df['Price'])
df['mileage_ihs'] = np.arcsinh(df['Mileage_Run'])
df['age'] = max(df['Make_Year'])-df['Make_Year']
df = df.drop(['Price','Mileage_Run','Make_Year','Unnamed: 0'],axis=1)
df.boxplot()
plt.show()
```


  2. Summarize the `Price` variable and create a kernel density plot. Use `.groupby()` and `.describe()` to summarize prices by brand (`Make`). Make a grouped kernel density plot by `Make`. Which car brands are the most expensive? What do prices look like in general?


```
price_summary = df0['Price'].describe()
print(price_summary)
price_kd = sns.kdeplot(data = df0, x='Price',hue='Make')
df0['Price'].groupby(df0['Make']).describe()
plt.show()
price_ihs_kd = sns.kdeplot(data=df,x='price_ihs',hue='Make')
df['price_ihs'].groupby(df['Make']).describe()
plt.show()
```
* MG Motor cars are most expensive, followed by Kia and Jeep. In general, prices range from 188,000 to 2,941,000 rupees. The average price is 741,019.5 rupees.


  3. Split the data into an 80% training set and a 20% testing set.
  ```
N = df.shape[0]
df = df.sample(frac=1, random_state=100)
train_size = int(.8*N)
df_train = df[0:train_size]
y_train = df_train['price_ihs']
df_test = df[train_size:]
y_test = df_test['price_ihs']
```

  4. Make a model where you regress price on the numeric variables alone; what is the $R^2$ and `RMSE` on the training set and test set? Make a second model where, for the categorical variables, you regress price on a model comprised of one-hot encoded regressors/features alone (you can use `pd.get_dummies()`; be careful of the dummy variable trap); what is the $R^2$ and `RMSE` on the test set? Which model performs better on the test set? Make a third model that combines all the regressors from the previous two; what is the $R^2$ and `RMSE` on the test set? Does the joint model perform better or worse, and by home much?


```
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Model 1: Numeric Variables Only
numeric_vars = ['age', 'mileage_ihs', 'Seating_Capacity']
X_train_numeric = df_train[numeric_vars]
X_test_numeric = df_test[numeric_vars]
regressor_numeric = LinearRegression().fit(X_train_numeric, y_train)
y_pred_numeric = regressor_numeric.predict(X_test_numeric)
print('Numeric only R-squared: ', regressor_numeric.score(X_test_numeric, y_test))
rmse_numeric = np.sqrt(mean_squared_error(y_test, y_pred_numeric))
print('Numeric only RMSE: ', rmse_numeric)

# Model 2: Categorical Variables Only
categorical_vars = ['Make', 'Body_Type', 'Color', 'Fuel_Type', 'Transmission', 'Transmission_Type']
dummies_train = pd.DataFrame()
dummies_test = pd.DataFrame()
for var in categorical_vars:
    dummies = pd.get_dummies(df[var], drop_first=True, dtype=int)
    dummies_train = pd.concat([dummies_train, dummies.iloc[:train_size]], axis=1)
    dummies_test = pd.concat([dummies_test, dummies.iloc[train_size:]], axis=1)
regressor_categorical = LinearRegression().fit(dummies_train, y_train)
y_pred_categorical = regressor_categorical.predict(dummies_test)
print('Categorical only R-squared: ', regressor_categorical.score(dummies_test, y_test))
rmse_categorical = np.sqrt(mean_squared_error(y_test, y_pred_categorical))
print('Categorical only RMSE: ', rmse_categorical)

# Model 3: Combined Numeric and Categorical Variables
X_train_combined = pd.concat([X_train_numeric, dummies_train], axis=1)
X_test_combined = pd.concat([X_test_numeric, dummies_test], axis=1)
regressor_combined = LinearRegression().fit(X_train_combined, y_train)
y_pred_combined = regressor_combined.predict(X_test_combined)
print('Combined R-squared: ', regressor_combined.score(X_test_combined, y_test))
rmse_combined = np.sqrt(mean_squared_error(y_test, y_pred_combined))
print('Combined RMSE: ', rmse_combined)
```
* Numeric only R-squared:  0.45254262356326824
Numeric only RMSE:  0.33392654735906463
Categorical only R-squared:  0.6298129532407464
Categorical only RMSE:  0.27459106425227275
Combined R-squared:  0.7999206763763922
Combined RMSE:  0.20187237686198908
* The joint model performs the best, for we see a significant improvement in R-squared and a reduction in RMSE.


  5. Use the `PolynomialFeatures` function from `sklearn` to expand the set of numerical variables you're using in the regression. As you increase the degree of the expansion, how do the $R^2$ and `RMSE` change? At what point does $R^2$ go negative on the test set? For your best model with expanded features, what is the $R^2$ and `RMSE`? How does it compare to your best model from part 4?

```
from sklearn.preprocessing import PolynomialFeatures

numeric_vars = ['age', 'mileage_ihs', 'Seating_Capacity']
categorical_vars = ['Make', 'Body_Type', 'Color', 'Fuel_Type', 'Transmission', 'Transmission_Type']

dummies = pd.get_dummies(df[categorical_vars], drop_first=True)
dummies_train = dummies.iloc[:train_size]
dummies_test = dummies.iloc[train_size:]

for degree in np.arange(1, 5):
    polynomial_expander = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_numeric_expanded = polynomial_expander.fit_transform(df_train[numeric_vars])
    X_test_numeric_expanded = polynomial_expander.transform(df_test[numeric_vars])
    expanded_feature_names = polynomial_expander.get_feature_names_out(numeric_vars)
    X_train_numeric_expanded_df = pd.DataFrame(X_train_numeric_expanded, columns=expanded_feature_names)
    X_test_numeric_expanded_df = pd.DataFrame(X_test_numeric_expanded, columns=expanded_feature_names)
    X_train_combined = pd.concat([X_train_numeric_expanded_df, dummies_train.reset_index(drop=True)], axis=1)
    X_test_combined = pd.concat([X_test_numeric_expanded_df, dummies_test.reset_index(drop=True)], axis=1)
    regressor_combined_expanded = LinearRegression().fit(X_train_combined, y_train)
    y_pred_combined_expanded = regressor_combined_expanded.predict(X_test_combined)
    r_squared_expanded = regressor_combined_expanded.score(X_test_combined, y_test)
    rmse_expanded = np.sqrt(mean_squared_error(y_test, y_pred_combined_expanded))
    print(f'Degree {degree} R-squared: {r_squared_expanded}')
    print(f'Degree {degree} RMSE: {rmse_expanded}')
```
* As degree increases from 1 to 2 and 2 to 3, R-squared and RMSE experience minor increases and decreases but generally stay around the same value. R-squared becomes negative at degree 4. The best degrees is 2 with a R-squared of 0.8025408094604638 and RMSE of 0.20054621389085775. This is just slightly better than our best model from part 4.

  6. For your best model so far, determine the predicted values for the test data and plot them against the true values. Do the predicted values and true values roughly line up along the diagonal, or not? Compute the residuals/errors for the test data and create a kernel density plot. Do the residuals look roughly bell-shaped around zero? Evaluate the strengths and weaknesses of your model.


```
best_degree = 2

polynomial_expander = PolynomialFeatures(degree=best_degree, include_bias=False)
X_train_numeric_expanded = polynomial_expander.fit_transform(df_train[numeric_vars])
X_test_numeric_expanded = polynomial_expander.transform(df_test[numeric_vars])
expanded_feature_names = polynomial_expander.get_feature_names_out(numeric_vars)
X_train_numeric_expanded_df = pd.DataFrame(X_train_numeric_expanded, columns=expanded_feature_names)
X_test_numeric_expanded_df = pd.DataFrame(X_test_numeric_expanded, columns=expanded_feature_names)
X_train_combined = pd.concat([X_train_numeric_expanded_df, dummies_train.reset_index(drop=True)], axis=1)
X_test_combined = pd.concat([X_test_numeric_expanded_df, dummies_test.reset_index(drop=True)], axis=1)
regressor_combined_expanded = LinearRegression().fit(X_train_combined, y_train)
y_pred_combined_expanded = regressor_combined_expanded.predict(X_test_combined)

residuals = y_test - y_pred_combined_expanded

# Scatter plot of true vs. predicted values
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred_combined_expanded)
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('True Values vs. Predicted Values')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)  # Diagonal line for reference
plt.show()

# Kernel density plot of residuals
plt.figure(figsize=(10, 6))
sns.kdeplot(residuals, shade=True)
plt.title('Kernel Density Plot of Residuals')
plt.xlabel('Residuals')
plt.ylabel('Density')
plt.show()

from sklearn import tree

X_train_numeric = df_train[numeric_vars].reset_index(drop=True)
X_train_categorical = dummies_train.reset_index(drop=True)
X_train_all = pd.concat([X_train_numeric, X_train_categorical], axis=1)

X_test_numeric = df_test[numeric_vars].reset_index(drop=True)
X_test_categorical = dummies_test.reset_index(drop=True)
X_test_all = pd.concat([X_test_numeric, X_test_categorical], axis=1)

best_depth = 0
best_rmse = float('inf')
best_r2 = float('-inf')

sup_depth = 20
for depth in np.arange(2, sup_depth):
    decision_tree_model = tree.DecisionTreeRegressor(max_depth=depth)
    decision_tree_model.fit(X_train_all, y_train)
    y_pred_tree = decision_tree_model.predict(X_test_all)
    rmse_tree = np.sqrt(mean_squared_error(y_test, y_pred_tree))
    r2_tree = decision_tree_model.score(X_test_all, y_test)
    print(f'Depth: {depth}, RMSE: {rmse_tree:.4f}, R-squared: {r2_tree:.4f}')

```
* The model is generally a good fit because predicted values align with true values across an upward trend, though the alignment deviates at higher values. We also see that the residuals center around zero in a bell-shaped distribution. However, there is somw right skewness and increased spread at higher values which may mean the model's tendency to underestimate larger prices.



**Q3.** This is a question about linear regression. The outcome is whether a defendant is held pre-trial in the Virginia justice system. We would like to understand how that outcome is predicted by characteristics of the defendant, particularly race. Let's be very careful/clear: We aren't saying anyone *should* be held without bond or asserting that people with different demographic variables *should* be more likely to be held, but instead trying to predict whether people with different characteristics *are empirically more likely* to be held without bond, given the available information. This is the first step we would take in investigating whether a system is fair, or how large the disparities are: Does it treat people with similar observable characteristics similarly, or not? We are going to look at a common question: Are Black defendants treated differently from white or Asian ones? (There are Native American defendants, but there are 11 in total, which is such a small number of observations that is difficult to clearly say anything about how this group is treated relative to the others.)

The variables in the data are:

  - `held_wo_bail`: Whether a defendant is held without bail before trial (Boolean logical)
  - `race`, `sex`: Categorical demographic variables
  - `is_poor`: Whether the defendant is classified as indigent
  - `prior_F`, `prior_M`: The number of prior felony and misdemeanor arrests
  - `case_type`: A categorical variable indicating a misdemeanor `M` or felony `F` or infraction `I` or special case `S`
  - `age`: Defendant's age
  - `bond`, `bond_NA`, `bond_type`: The amount of any bond, whether it is missing, and the type
  - `sentence`, `sentence_NA`, `sentence_type`: The length of any sentence, whether it is missing, and the type

1. Load the `pretrial_data.csv` data. Notice that there are `nan`s, but the data are relatively clean. Because there are `.nan`s among variables you won't use, you'll want to narrow down your analysis to the relevant variables before dropping or imputing missing values.
2. Create a dummy variable indicating that the defendant is Black.
3. Regress `held` on `Black`. What is the slope coefficient Interpret the coefficient on the Black dummy variable: How much more likely is a black person to be held without bail? What is the $R^2$ of the model?
4. Before doing this question, please think for a few minutes about how to make the process of running the following regressions as efficient as possible, before jumping into writing code. Repeat part 2, for the following specifications, keeping track of the coefficient on the Black dummy variable each time:
      - `held` on `Black` and `sex`
      - `held` on `Black` and `sex` and `is_poor`
      - `held` on `Black` and `sex` and `is_poor` and `prior_F`
      - `held` on `Black` and `sex` and `is_poor` and `prior_F` and `case_type`
What happens to the coefficient on the Black dummy variable as you include more regressors/features/controls in the regression? Explain your findings.
5. Suppose we don't want to see just `Black` and `sex`, but `Black` interacted with `sex`: Are Black men and Black women treated systemically differently from the rest of the population? Implement this in a regression, and explain your findings.
6. Imagine someone argued we should use these kinds of models to help a judge or magistrate make bail decisions (you could obviously go back and make this kind of model for the bond and sentence variables, then deploy it on new cases to predict what their bond and sentence values would be). What concerns would you have? Do you think society should be using data-driven and automated tools like that? Explain your concerns clearly.

**Q4.** This is a math question to review the derivation of the OLS estimator (but only if you are into that kind of thing!). We are going to do it slightly differently from what we did in class, though. We will use a linear predictor and minimize the Sum of Squared Errors, just as in class. But, we are going to de-mean $X$ first, creating another variable $z_i = x_i - \bar{x}$ where
$$
\bar{x} = \dfrac{1}{N} \sum_{i=1}^N x_i,
$$
so the model is $\hat{y}_i = a + b z_i$ and the `SSE` is
$$
\text{SSE}(a,b) = \sum_{i=1}^N (y_i - a - bz_i)^2.
$$

  1. Take partial derivatives of the `SSE` with respect to $a$ and $b$. You should get

\begin{alignat*}{3}
\sum_{i=1}^N -2(y_i - a- bz_i) &=& 0 \\
\sum_{i=1}^N -2(y_i - a - bz_i)z_i &=& 0.
\end{alignat*}

  2. Solve for the solutions to the above equations. Big hint: $\bar{z} = 0$, since we subtracted the mean of $x$ from $x$ to get $z$. You should get

\begin{alignat*}{3}
a^* &=& \bar{y} \\
b^* &=& \dfrac{\sum_{i=1}^N(y_i - \bar{y})z_i}{\sum_{i=1}^N z_i^2}.
\end{alignat*}

  3. Substitute $z_i = x_i - \bar{x}$ back into the above equations. You should get
  
\begin{alignat*}{3}
a^* &=& \bar{y} \\
b^* &=& \dfrac{\sum_{i=1}^N(y_i - \bar{y})(x_i-\bar{x})}{\sum_{i=1}^N (x_i-\bar{x})^2},
\end{alignat*}

which can be written in terms of sample covariance and sample variance as:

\begin{alignat*}{3}
a^* &=& \bar{y} \\
b^* &=& \dfrac{\text{cov}(x,y)}{\text{var}(x)}.
\end{alignat*}

This is typically the preferred way of expressing the OLS coefficients.

4. When will $b^*$ be large or small, depending on the relationship between $x$ and $y$ and the amount of "noise"/variance in $x$? What does $a^*$ represent?
5. Suppose you have measurement error in $x$ which artificially inflates its variance (e.g. bad data cleaning). What happens to the $b^*$ coefficient? How will affect your ability to predict? (This phenomenon is called **attenuation**.)

**Q5.**
1. Find a dataset on a topic you're interested in. Some easy options are data.gov, kaggle.com, and data.world.
2. Clean the data and do some exploratory data analysis on key variables that interest you. Pick a particular target/outcome variable and features/predictors.
3. Split the sample into an ~80% training set and a ~20% test set.
4. Run a few regressions of your target/outcome variable on a variety of features/predictors. Compute the SSE on the test set.
5. Which model performed the best, and why?
6. What did you learn?